In [2]:
from CBAStats.Player import *
from CBAStats.Team import *
from CBAStats.Player import stats_output
import datetime
from Scraping import Scraper
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import pymysql

In [3]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-01-25 20:26:25.636953


# 更新数据

In [4]:
schedule_output = f'../StatsData/GamesSchedulePage_{datetime.date.today()}.csv'
game_stats_output = f'../StatsData/All_Games_Stats_{datetime.date.today()}.csv'
teams_stats_output = f'../StatsData/TeamsAdvancedStats_{datetime.date.today()}.csv'
players_stats_output = f'../StatsData/PlayersAdvancedStats_{datetime.date.today()}.csv'

for output in [schedule_output,game_stats_output,teams_stats_output,players_stats_output]:
    path = Path(output)
    print(f'Checking {output}.')
    # check if folder is reachable
    assert (path.parent.exists()),f"Folder '{path.parent}' does not exist."
    # check if file already exists
    if path.exists():
        print(f'\tFile {path} already exists! It will be overwritten.')
    print('\tPass!')

Checking ../StatsData/GamesSchedulePage_2020-01-25.csv.
	File ../StatsData/GamesSchedulePage_2020-01-25.csv already exists! It will be overwritten.
	Pass!
Checking ../StatsData/All_Games_Stats_2020-01-25.csv.
	File ../StatsData/All_Games_Stats_2020-01-25.csv already exists! It will be overwritten.
	Pass!
Checking ../StatsData/TeamsAdvancedStats_2020-01-25.csv.
	Pass!
Checking ../StatsData/PlayersAdvancedStats_2020-01-25.csv.
	Pass!


In [5]:
# only supports csv file for now
Scraper.scrape_sina_schedule(output_path=schedule_output)
Scraper.scrape_game_details(input_file=schedule_output,
                            output_file=game_stats_output)

开始爬取所有赛程、数据统计链接.
所有赛程、数据统计链接将会存在 ../StatsData/GamesSchedulePage_2020-01-25.csv.
所有赛程、数据统计链接已保存在 ../StatsData/GamesSchedulePage_2020-01-25.csv.
从../StatsData/GamesSchedulePage_2020-01-25.csv读取所有赛程、数据统计链接
读取完毕
抓取比赛  2019202000001   广东   辽宁   2020-01-25 20:26:37.915361
抓取比赛  2019202000002   江苏   浙江   2020-01-25 20:26:40.795940


KeyboardInterrupt: 

# 载入爬取的数据

In [6]:
df = pd.read_csv(game_stats_output,dtype={'号码':'str','Game_ID':'str'})
df

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,得分,Game_ID,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投
0,布鲁克斯,6,36,1,0,3,9,3,2,1,...,27,2019202000001,广东,辽宁,8,10,2,10,5,5
1,任骏飞,20,30,1,1,1,1,3,3,0,...,19,2019202000001,广东,辽宁,5,8,3,6,0,0
2,易建联,9,35,1,1,11,2,2,1,1,...,16,2019202000001,广东,辽宁,7,8,0,0,2,4
3,赵睿,10,31,1,2,5,4,3,2,0,...,10,2019202000001,广东,辽宁,5,9,0,3,0,1
4,胡明轩,3,13,1,0,1,0,1,0,1,...,3,2019202000001,广东,辽宁,0,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10541,张皓嘉,25,0,0,0,0,0,0,0,0,...,0,2019202000300,广东,新疆,0,0,0,0,0,0
10542,苏伟,12,0,0,0,0,0,0,0,0,...,0,2019202000300,广东,新疆,0,0,0,0,0,0
10543,徐杰,2,0,0,0,0,0,0,0,0,...,0,2019202000300,广东,新疆,0,0,0,0,0,0
10544,方渭博,1,0,0,0,0,0,0,0,0,...,0,2019202000300,广东,新疆,0,0,0,0,0,0


# Insert Records

In [7]:
from sqlalchemy import create_engine
import pymysql

user_name = 'master'
passcode = 'Pw#cbashuju0131'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
connection= engine.connect()

In [8]:
df.to_sql(name='PlayerStatsPerGame',con=connection,index=False,if_exists='replace')

In [9]:
connection.close()